# Comparison Using LFW Dataset - VGGFace (VGG16)

Several feature extractor:
1. Face Embedding: Facenet
2. Face Embedding: VGG Face
3. Face Embedding: VGG Face - VGG16
4. Face Embedding: VGG Face - RESNET50
5. LBPH (Local Binary Pattern Histogram)

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Read Dataset

In [2]:
import os

In [3]:
DIR_PATH = '../lfw/'
image_path_list = []
labels = []
name_dictionary = {}
counter = 0
for root, dirs, files in os.walk(DIR_PATH):
    for filename in files:
        person_name = ' '.join(filename.split('.')[0].split('_')[0:-1]) 
        file_path = os.path.join(root, filename)
        if person_name not in name_dictionary:
            counter += 1
            name_dictionary[person_name] = counter
        image_path_list.append(file_path)
        labels.append(name_dictionary[person_name])

In [4]:
print(len(labels))
print(len(image_path_list))

13233
13233


In [5]:
image_path_list[0:5]

['../lfw/Ryan_Newman/Ryan_Newman_0001.jpg',
 '../lfw/Dimitar_Berbatov/Dimitar_Berbatov_0001.jpg',
 '../lfw/Ed_Rendell/Ed_Rendell_0001.jpg',
 '../lfw/Joe_Crede/Joe_Crede_0001.jpg',
 '../lfw/Norman_Mailer/Norman_Mailer_0001.jpg']

## Shuffle It

In [6]:
import random

In [7]:
temp = list(zip(image_path_list, labels))
random.Random(0).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [8]:
image_path_list[0:5]

('../lfw/Tommy_Maddox/Tommy_Maddox_0001.jpg',
 '../lfw/David_Millar/David_Millar_0001.jpg',
 '../lfw/Gregg_Popovich/Gregg_Popovich_0004.jpg',
 '../lfw/Shimon_Peres/Shimon_Peres_0001.jpg',
 '../lfw/Rudolph_Giuliani/Rudolph_Giuliani_0024.jpg')

In [9]:
labels[0:5]

(3038, 1517, 3115, 834, 1016)

In [10]:
name_dictionary['John Ashcroft']

482

# Experiment

In [11]:
FACENET_MODEL_PATH = 'model_data/facenet/20180402-114759'
VGGFACE_MODEL_PATH = 'model_data/vgg_face_weights.h5'

In [12]:
mtcnn_detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
def predict(image_representation_list, true_label, test_representation, min_distance):
    minimum_label = None
    minimum_distance = min_distance
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [14]:
def compute_score(predictions, labels):
    count_same = 0
    for idx, prediction in enumerate(predictions):
        if labels[idx] == prediction:
            count_same += 1
    return count_same / len(predictions)

## VGGFace - VGG16

Based on experiment VGGFace - VGG16 have distance below 95 for euclidean if two image are the same person

In [15]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_vgg16')

### Testing Starts Here

In [16]:
THRESHOLD = 95

In [17]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [18]:
%%time
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
CPU times: user 4h 41min 24s, sys: 1h 12min 51s, total: 5h 54min 16s
Wall time: 1h 1min 17s


In [19]:
feature_extractor.extract_image(img)

array([7.10956514e-01, 6.23499870e-01, 2.00736523e+00, 2.21811786e-01,
       1.10849090e-01, 1.34789670e+00, 3.71814394e+00, 4.20973569e-01,
       4.89456058e-01, 5.40997458e+00, 1.41513538e+00, 3.39693522e+00,
       3.96513772e+00, 4.52590561e+00, 3.52742732e-01, 0.00000000e+00,
       1.81685376e+00, 3.15170377e-01, 4.03955841e+00, 3.87913322e+00,
       2.65097618e+00, 0.00000000e+00, 5.49662924e+00, 1.34617674e+00,
       0.00000000e+00, 2.08477211e+00, 1.64396811e+00, 5.60633469e+00,
       4.18782085e-01, 1.59958565e+00, 8.65523458e-01, 1.50462985e-01,
       0.00000000e+00, 2.11442852e+00, 5.56344795e+00, 4.31068778e-01,
       8.59199703e-01, 9.23207581e-01, 2.05308700e+00, 3.42654490e+00,
       4.07599735e+00, 4.58178854e+00, 0.00000000e+00, 1.99404061e+00,
       0.00000000e+00, 1.39009148e-01, 1.81344724e+00, 9.82851148e-01,
       4.52302742e+00, 1.75863862e+00, 3.60591626e+00, 1.95506632e+00,
       2.73243380e+00, 3.08880186e+00, 1.14372447e-02, 2.66136169e+00,
      

In [20]:
image_path_list[37]

'../lfw/George_W_Bush/George_W_Bush_0233.jpg'

In [21]:
compute_score(prediction_result, labels)

0.3436106702939621

In [22]:
len(prediction_result)

13233

In [23]:
len(labels)

13233

In [24]:
# feature_extractor.extractor.close_session()

In [25]:
image_path_list = sorted(image_path_list)

In [26]:
image_path_list[31:50]

['../lfw/Abdullah_Gul/Abdullah_Gul_0001.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0002.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0003.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0004.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0005.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0006.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0007.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0008.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0009.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0010.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0011.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0012.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0013.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0014.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0015.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0016.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0017.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0018.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0019.jpg']

In [27]:
%%time
for idx, image_path in enumerate(image_path_list[31:32]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
CPU times: user 1.22 s, sys: 336 ms, total: 1.55 s
Wall time: 249 ms


In [28]:
base_feature = feature_test.copy()
base_feature

array([4.65718478e-01, 7.17355919e+00, 1.02863121e+00, 3.69372582e+00,
       0.00000000e+00, 5.61206520e-01, 2.21609616e+00, 3.97615433e+00,
       1.13573246e+01, 1.92073703e+00, 3.36199856e+00, 2.14290071e+00,
       6.93783617e+00, 3.55302334e+00, 1.44352703e+01, 7.30758619e+00,
       6.03733718e-01, 7.17423964e+00, 1.94288135e-01, 4.22098112e+00,
       5.83752453e-01, 2.74037290e+00, 9.79675865e+00, 2.27779579e+00,
       0.00000000e+00, 8.42406034e-01, 7.48322248e-01, 2.18280271e-01,
       1.02587938e+01, 2.79036582e-01, 5.75377989e+00, 1.28302937e+01,
       1.83160210e+00, 9.86547756e+00, 1.32050447e+01, 7.49301314e-01,
       3.91818428e+00, 1.26217842e-01, 1.14301097e+00, 5.35817623e-01,
       5.88977098e-01, 8.46186924e+00, 4.31389809e+00, 8.48773956e+00,
       2.14935184e+00, 9.26305473e-01, 3.10160017e+00, 0.00000000e+00,
       4.73534393e+00, 3.64004135e+00, 2.50589561e+00, 0.00000000e+00,
       9.40837622e-01, 1.50598609e+00, 1.25661302e+00, 1.68752408e+00,
      

In [29]:
%%time
for idx, image_path in enumerate(image_path_list[31:50]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)

Checkpoint 0
0.0
95.87703704833984
84.3180160522461
101.34901428222656
81.51607513427734
72.37901306152344
64.24150085449219
83.32807159423828
117.56531524658203
90.58826446533203
64.10580444335938
88.19580841064453
70.6436996459961
73.2634506225586
78.26561737060547
90.03889465332031
72.2740478515625
100.68028259277344
80.21116638183594
CPU times: user 40.5 s, sys: 5.94 s, total: 46.4 s
Wall time: 7.11 s


In [30]:
%%time
for idx, image_path in enumerate(image_path_list[0:30]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)

Checkpoint 0
135.57504272460938
146.82711791992188
132.0380401611328
116.71697998046875
108.9965591430664
115.4239730834961
114.09256744384766
119.49925231933594
127.69943237304688
120.90898132324219
129.04684448242188
112.8216781616211
106.85029602050781
123.42777252197266
118.61515808105469
109.36315155029297
113.8707275390625
119.15155792236328
127.2124252319336
125.74362182617188
132.28395080566406
126.56423950195312
142.0778045654297
110.94503784179688
145.28713989257812
118.19072723388672
118.54680633544922
116.43965911865234
121.696044921875
118.87250518798828
CPU times: user 56.1 s, sys: 12 s, total: 1min 8s
Wall time: 11.1 s
